In [85]:
import pandas as pd
import mariadb
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.ensemble import *
import numpy as np
import bisect
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns

conn  = mariadb.connect(
    user='team3', 
    password='dbdb', 
    database='team3db', 
    host='localhost',
    port=3306
)

# 위 커넥션 정보와 동일하게 입력
engine = create_engine("mysql://{user}:{pw}@{host}/{db}".format(user='team3', pw='dbdb',host='localhost', db='team3db'))

In [86]:
select_sql = "SELECT * FROM train_UL"
df = pd.read_sql_query(select_sql, engine,index_col='index')
df.reset_index(inplace=True,drop=True)
df.drop(columns='SAMPLE_ID',inplace=True)
df

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,...,LENGTH,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE,CI_HOUR
0,SG,GIW5,Container,30.881018,2018-12-17 21:29,Z618338,30.0,24,24300,10.0,...,180.0,CQSB78,Panama,-0.527582,-0.075277,27.637513,2.014493,5,0.002615,3.450000
1,IN,UJM2,Bulk,0.000000,2014-09-23 6:59,X886125,30.0,13,35900,10.0,...,180.0,SPNO34,Marshall Islands,2.244962,0.213273,28.263135,3.102933,12,0.000217,0.000000
2,CN,EUC8,Container,0.000000,2015-02-03 22:00,T674582,50.0,12,146000,30.0,...,370.0,FNPK22,Malta,-1.050854,-0.122173,17.651842,3.210685,6,0.001614,0.000000
3,JP,ZAG4,Container,0.000000,2020-01-17 4:02,Y847238,20.0,18,6910,10.0,...,120.0,PBZV77,Bahamas,-3.180000,-1.610000,6.700000,2.629350,13,0.000356,0.000000
4,SG,GIW5,Container,27.037650,2020-01-26 7:51,A872328,50.0,10,116000,20.0,...,300.0,GUCE76,Liberia,-0.330000,-3.280000,25.600000,2.495953,15,0.002615,122.943750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391928,JP,QYY1,Container,0.000000,2017-06-06 5:02,Y375615,20.0,27,6820,10.0,...,110.0,KEJZ24,"China, People's Republic Of",0.836331,-0.767929,17.033054,2.157586,14,0.000552,0.000000
391929,SG,GIW5,Bulk,5.884603,2019-10-16 0:36,K635567,10.0,12,3160,10.0,...,80.0,JLTM64,Vietnam,-0.660000,0.970000,27.300000,1.253491,8,0.002615,122.943750
391930,US,QGN3,Bulk,64.231480,2021-03-23 22:35,J284147,30.0,8,60300,20.0,...,200.0,YERJ68,Singapore,-3.440000,4.035000,21.100000,4.766257,18,0.000155,41.482222
391931,TW,JWI3,Container,9.448179,2015-01-08 7:15,J644215,30.0,29,23800,10.0,...,170.0,HCZK58,Comoros,-0.996649,-0.826293,25.702965,2.987039,15,0.000990,7.485278


In [87]:
df.isnull().sum()

ARI_CO                0
ARI_PO                0
SHIP_TYPE_CATEGORY    0
DIST                  0
ATA                   0
ID                    0
BREADTH               0
BUILT                 0
DEADWEIGHT            0
DEPTH                 0
DRAUGHT               0
GT                    0
LENGTH                0
SHIPMANAGER           0
FLAG                  0
U_WIND                0
V_WIND                0
AIR_TEMPERATURE       0
BN                    0
ATA_LT                0
PORT_SIZE             0
CI_HOUR               0
dtype: int64

In [88]:
df_nation = df.groupby('ARI_CO')['CI_HOUR'].std().to_dict()
df_port = df.groupby('ARI_PO')['CI_HOUR'].std()
df_ship = df.groupby('SHIP_TYPE_CATEGORY')['CI_HOUR'].std()
df_flag = df.groupby('FLAG')['CI_HOUR'].std()

In [89]:
df['FLAG'].isnull().sum()

0

In [90]:
for port, mean_temp in df_nation.items():
    df.loc[df['ARI_CO']==port, 'ARI_CO'] = mean_temp
    

In [91]:
df=df.astype({'ARI_CO':'float'})

In [92]:
df['ARI_CO']

0         46.041829
1         41.255277
2         39.775467
3         37.839062
4         46.041829
            ...    
391928    37.839062
391929    46.041829
391930    46.265763
391931    29.296775
391932    29.296775
Name: ARI_CO, Length: 391933, dtype: float64

In [94]:
for port, mean_temp in df_port.items():
    df.loc[df['ARI_PO']==port, 'ARI_PO'] = mean_temp
df=df.astype({'ARI_PO':'float'})

In [95]:
for port, mean_temp in df_ship.items():
    df.loc[df['SHIP_TYPE_CATEGORY']==port, 'SHIP_TYPE_CATEGORY'] = mean_temp
df=df.astype({'SHIP_TYPE_CATEGORY':'float'})

In [96]:
for port, mean_temp in df_flag.items():
    df.loc[df['FLAG']==port, 'FLAG'] = mean_temp
df=df.astype({'FLAG':'float'})

In [97]:
df['FLAG'].isnull().sum()

9

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391933 entries, 0 to 391932
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ARI_CO              391933 non-null  float64
 1   ARI_PO              391933 non-null  float64
 2   SHIP_TYPE_CATEGORY  391933 non-null  float64
 3   DIST                391933 non-null  float64
 4   ATA                 391933 non-null  object 
 5   ID                  391933 non-null  object 
 6   BREADTH             391933 non-null  float64
 7   BUILT               391933 non-null  int64  
 8   DEADWEIGHT          391933 non-null  int64  
 9   DEPTH               391933 non-null  float64
 10  DRAUGHT             391933 non-null  float64
 11  GT                  391933 non-null  int64  
 12  LENGTH              391933 non-null  float64
 13  SHIPMANAGER         391933 non-null  object 
 14  FLAG                391924 non-null  float64
 15  U_WIND              391933 non-nul

In [99]:
df.isnull().sum()

ARI_CO                0
ARI_PO                0
SHIP_TYPE_CATEGORY    0
DIST                  0
ATA                   0
ID                    0
BREADTH               0
BUILT                 0
DEADWEIGHT            0
DEPTH                 0
DRAUGHT               0
GT                    0
LENGTH                0
SHIPMANAGER           0
FLAG                  9
U_WIND                0
V_WIND                0
AIR_TEMPERATURE       0
BN                    0
ATA_LT                0
PORT_SIZE             0
CI_HOUR               0
dtype: int64

In [100]:
df['ATA'] = pd.to_datetime(df['ATA'])
df['year'] =df['ATA'].dt.year
df['month'] =df['ATA'].dt.month
df['day'] =df['ATA'].dt.day
df['hour'] =df['ATA'].dt.hour
df['minute'] =df['ATA'].dt.minute
df['weekday'] =df['ATA'].dt.weekday

df.drop(columns='ATA',inplace=True)

In [101]:
df['FLAG'].fillna(df['FLAG'].mean(),inplace=True)

In [110]:
X = df[['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'DIST', 'BREADTH', 'BUILT', 'DEADWEIGHT', 'DEPTH', 'DRAUGHT', 'GT', 'LENGTH', 
            'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'PORT_SIZE', 'year', 'weekday','BN', 'ATA_LT', 'month', 'day', 'hour', 'minute']]
y = df['CI_HOUR']

In [104]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391933 entries, 0 to 391932
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ARI_CO              391933 non-null  float64
 1   ARI_PO              391933 non-null  float64
 2   SHIP_TYPE_CATEGORY  391933 non-null  float64
 3   DIST                391933 non-null  float64
 4   BREADTH             391933 non-null  float64
 5   BUILT               391933 non-null  int64  
 6   DEADWEIGHT          391933 non-null  int64  
 7   DEPTH               391933 non-null  float64
 8   DRAUGHT             391933 non-null  float64
 9   GT                  391933 non-null  int64  
 10  LENGTH              391933 non-null  float64
 11  FLAG                391933 non-null  float64
 12  U_WIND              391933 non-null  float64
 13  V_WIND              391933 non-null  float64
 14  AIR_TEMPERATURE     391933 non-null  float64
 15  PORT_SIZE           391933 non-nul

In [111]:
import statsmodels.api as sm

# 독립 변수에 상수항 추가
X_with_const = sm.add_constant(X)

# 회귀 모델 생성
model = sm.OLS(y, X_with_const)

# 모델 피팅
results = model.fit()

# 결과 요약
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                CI_HOUR   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     9033.
Date:                Fri, 05 Apr 2024   Prob (F-statistic):               0.00
Time:                        16:46:15   Log-Likelihood:            -1.9521e+06
No. Observations:              391933   AIC:                         3.904e+06
Df Residuals:                  391909   BIC:                         3.904e+06
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
ARI_CO                 0.8216      0

In [112]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

df_vif = pd.DataFrame()
df_vif['VIF'] = np.round([variance_inflation_factor(X.values,i) for i in range(X.shape[1])],3)
df_vif['feature'] = X.columns
df_vif.sort_values(by='VIF',ascending=False)

,VIF,feature
8,805679.544,DRAUGHT
9,30.732,GT
6,21.000,DEADWEIGHT
10,17.740,LENGTH
4,7.880,BREADTH
7,4.018,DEPTH
2,2.857,SHIP_TYPE_CATEGORY
1,1.695,ARI_PO
0,1.652,ARI_CO
15,1.567,PORT_SIZE


In [117]:
df.boxplot(column=['DRAUGHT'])

<Axes: >

In [106]:
# df.to_csv('./data/train_std.csv')

In [107]:
X = df[['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'DIST', 'BREADTH', 'BUILT', 'DEADWEIGHT', 'DEPTH', 'DRAUGHT', 'GT', 'LENGTH', 
            'FLAG',  'V_WIND', 'AIR_TEMPERATURE', 'PORT_SIZE', 'year', 'weekday', 'ATA_LT', 'month']]
y = df['CI_HOUR']

In [108]:
# 독립 변수에 상수항 추가
X_with_const = sm.add_constant(X)

# 회귀 모델 생성
model = sm.OLS(y, X_with_const)

# 모델 피팅
results = model.fit()

# 결과 요약
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                CI_HOUR   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                 1.154e+04
Date:                Fri, 05 Apr 2024   Prob (F-statistic):               0.00
Time:                        16:39:43   Log-Likelihood:            -1.9521e+06
No. Observations:              391933   AIC:                         3.904e+06
Df Residuals:                  391914   BIC:                         3.904e+06
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
ARI_CO                 0.8224      0

In [109]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

df_vif = pd.DataFrame()
df_vif['VIF'] = np.round([variance_inflation_factor(X.values,i) for i in range(X.shape[1])],3)
df_vif['feature'] = X.columns
df_vif.sort_values(by='VIF',ascending=False)

,VIF,feature
8,794909.092,DRAUGHT
9,30.619,GT
6,20.945,DEADWEIGHT
10,17.703,LENGTH
4,7.869,BREADTH
7,4.016,DEPTH
2,2.854,SHIP_TYPE_CATEGORY
1,1.687,ARI_PO
0,1.644,ARI_CO
14,1.551,PORT_SIZE
